# Домашнее задание №1
Хорин Роман ИАД-2

Необходимо проверить следующую гипотезу: в текстах разных стилей частоты речи имеют разные характеры распределений.

In [1]:
import pymystem3 as stm
import pymorphy2
import nltk
from nltk.tokenize import WhitespaceTokenizer
from string import punctuation
from nltk.corpus import stopwords
from pymystem3 import Mystem
from scipy import stats

morph = pymorphy2.MorphAnalyzer()

## Основные методы

В данном разделе представлены основные методы, использующиеся в рамках данного домашнего задания.

In [2]:
# 'ADJF' - прилагательное (полное)
# 'ADJS' - прилагательное (краткое)
# 'ADVB' - наречие
# 'COMP' - компаратив (сравнение)
# 'CONJ' - союз
# 'GRND' - деепричастие
# 'INFN' - инфинитив
# 'INTJ' - междометие
# 'NOUN' - существительное
# 'NPRO' - местомение-существительное
# 'NUMR' - числительное
# 'PRCL' - частица
# 'PRED' - предикатив
# 'PREP' - предлог
# 'PRTF' - причастие (полное)
# 'PRTS' - причастие (короткое)
# 'VERB' - глагол

In [3]:
"""Метод формирует список частей речи, имеющихся в тексте"""

def Gain_POSes(types):
    parts_of_speech = []
    for t in types:
        try:
            gram_info = morph.parse(t)[0]
            POS = gram_info.tag.POS # определяем часть речи слова
            if POS in parts_of_speech:
                continue
            else:
                parts_of_speech.append(POS) # добавляем часть речи в список, если она не встречалась в нем раньше
        except:
            pass
    try:
        parts_of_speech.remove(None) # удаляем None
    except:
        pass
    return parts_of_speech

In [4]:
"""Метод считает количество слов определенной части речи в тексте"""

def Count_part_of_the_speech(text_types, part_name):
    part_dict = nltk.FreqDist()
    for t in text_types:
        try:
            gram_info = morph.parse(t)[0] # парсим слово
            if part_name == gram_info.tag.POS: # проверям на равенство части речи         
                part_dict[t] += text_types[t] # добавляем в словарь
        except:
            pass
    return sum(part_dict.values()) # возращаем количество слов, соответствующее части речи

In [5]:
"""Метод формирует частотный словарь частей речи текста"""

def Generate_pos_freq_dic(types, parts_of_speech):
    freq = nltk.FreqDist()
    for pos in parts_of_speech: 
        # для каждой части речи определяем количество слов, ей соответствующее, и добавляем количество в частотный словарь
        freq[pos] = Count_part_of_the_speech(types, pos) 
    return freq

## Основная часть работы

В данной части работы проводится основная работа с текстами (их очистка и токенизация), составляются частотный словари (часть речи-количество слов) для каждого из текстов, а также считается коэффициент корреляции Спирмена между составленными частотными словарями частей речи.

#### 1. Считываем коллекцию текстов двух разных стилей, считаем количество токеном и типов в каждой коллекции.


In [28]:
infile_publ = 'publicism.txt'
infile_art = 'art.txt'

# исключающая пунктуация, цифры и знаки
exclude = set(punctuation + '0123456789' + u'-' + u'—'+u'–—'+u'«»'+'\n'+ u'\ufeff')

# считываем тексты
text_publ = open(infile_publ, 'r', encoding='utf-8').read()
text_art = open(infile_art, 'r', encoding='utf-8').read()


# проводим предварительную очистку текстов
text_publ = ''.join(ch for ch in text_publ if ch not in exclude)
text_art = ''.join(ch for ch in text_art if ch not in exclude)

# Считаем количество токенов и типов в текстах
# токен - слово + его место в тексте
# тип - словоформа

tokens_publ = WhitespaceTokenizer().tokenize(text_publ.lower())
types_publ = nltk.FreqDist(tokens_publ)
print('Publicistic text with %d tokens and %d types. \n'%(len(tokens_publ), len(types_publ)))

tokens_art = WhitespaceTokenizer().tokenize(text_art.lower())
types_art = nltk.FreqDist(tokens_art)
print('Artistic text with %d tokens and %d types.'%(len(tokens_art), len(types_art)))

Publicistic text with 5944 tokens and 3003 types. 

Artistic text with 5848 tokens and 3090 types.


#### 2. При помощи nltk.FreqDist() и морфологического процессора pymorphy2 составляем частоные словари: часть речи - количество слов, к ней относящихся.

Формируем список частей речи, используя метод Gain_POSes()

In [29]:
parts_of_speech_publ = Gain_POSes(types_publ)
parts_of_speech_art = Gain_POSes(types_art)

# общий список частей речи представляет из себя объединение множеств частей речи из обоих текстов
parts_of_speech = list(set.union(set(parts_of_speech_publ), set(parts_of_speech_art)))
parts_of_speech

['NOUN',
 'ADJF',
 'ADJS',
 'VERB',
 'NPRO',
 'PRED',
 'COMP',
 'CONJ',
 'NUMR',
 'PREP',
 'PRTF',
 'INTJ',
 'ADVB',
 'PRTS',
 'GRND',
 'INFN',
 'PRCL']

Формируем частотные словари частей речи для обоих текстов, используя метод Generate_pos_freq_dic().

In [30]:
%%time
pos_freq_dic_publ = Generate_pos_freq_dic(types_publ, parts_of_speech)
pos_freq_dic_art = Generate_pos_freq_dic(types_art, parts_of_speech)

Wall time: 36.3 s


Частотный словарь частей речи для художественного текста.

In [31]:
print('Artistic text POS frequency dictionary:')
pos_freq_dic_art

Artistic text POS frequency dictionary:


FreqDist({'ADJF': 936,
          'ADJS': 80,
          'ADVB': 353,
          'COMP': 13,
          'CONJ': 540,
          'GRND': 74,
          'INFN': 157,
          'INTJ': 6,
          'NOUN': 1790,
          'NPRO': 283,
          'NUMR': 19,
          'PRCL': 298,
          'PRED': 23,
          'PREP': 613,
          'PRTF': 152,
          'PRTS': 38,
          'VERB': 463})

Частотный словарь частей речи для публицистического текста.

In [32]:
print('Publicistic text POS frequency dictionary:')
pos_freq_dic_publ

Publicistic text POS frequency dictionary:


FreqDist({'ADJF': 744,
          'ADJS': 33,
          'ADVB': 248,
          'COMP': 19,
          'CONJ': 461,
          'GRND': 14,
          'INFN': 143,
          'INTJ': 9,
          'NOUN': 2241,
          'NPRO': 201,
          'NUMR': 34,
          'PRCL': 175,
          'PRED': 21,
          'PREP': 778,
          'PRTF': 87,
          'PRTS': 63,
          'VERB': 587})

#### 3. Считаем коэффициент корреляции Спирмена между сформированными частотными словарями.

In [33]:
corrcoef = stats.spearmanr(list(pos_freq_dic_art.values()), list(pos_freq_dic_publ.values()))[0]
print('Spearman correlation between two texts:',corrcoef)

Spearman correlation between two texts: 0.950980392157


Коэффициент корреляции Спирмена оказался 0,95. Это означает, что по частотной составляющей тексты вполне схожи друг с другом, но имеют некоторые отличия. Так в художественном тексте больше кратких и полных прилагательных, а также полных причастий, чем в публицистическом. В публицистическом тексте же больше глаголов, существительных и предлогов, чем в художественном.